# Next Word prediction

# using Lstm

In [43]:
import pandas as pd

In [44]:
import nltk
nltk.download("gutenberg")

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\vasud\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [45]:
from nltk.corpus import gutenberg
data=gutenberg.raw("shakespeare-hamlet.txt")

with open("shakespear_hamlet.txt",'w')as file:
    file.write(data)

In [46]:
import tensorflow

In [47]:
#data pre preprosesing

import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
#load the data
with open ('shakespear_hamlet.txt','r') as file:
    text=file.read().lower()

In [48]:
#convert into token
tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])

In [49]:
#Total number of words in Text
total_word=len(tokenizer.word_index)
total_word

4817

In [50]:
#to find unique word

In [51]:
#Create input sequence
input_seq=[]
for line in text.split('\n'):
    token_list=tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence=token_list[:i+1]
        input_seq.append(n_gram_sequence)

In [52]:
#max sequence
max_seq_len = max(len(x) for x in input_seq)
input_sequences= np.array (pad_sequences(input_seq,maxlen=max_seq_len,padding="pre"))

In [53]:
input_sequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]])

In [54]:
x=input_sequences[:,:-1]
y=input_sequences[:,-1]

In [55]:
x

array([[   0,    0,    0, ...,    0,    0,    1],
       [   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       ...,
       [   0,    0,    0, ...,  687,    4,   45],
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4]])

In [56]:
y

array([ 687,    4,   45, ..., 1047,    4,  193])

In [57]:
from tensorflow.keras.utils import  to_categorical

y=to_categorical(y,num_classes=total_word)

In [58]:
print(y)
print(len(y[0]))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
4817


In [59]:
# split the data into traning and testing 
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25)

In [60]:
#Early stooping
from tensorflow.keras.callbacks import EarlyStopping
earlystopping=EarlyStopping(monitor='val_loss',
                            patience=3,
                            restore_best_weights=True
                            )

In [63]:
# Model training
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense,Embedding,LSTM,Flatten,Dropout

model= Sequential()
model.add(Embedding(total_word+1,1000,
                    input_length=max_seq_len
                    ))
model.add(LSTM(150,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_word,activation="softmax"))


d:\btch 16\NPl\.venv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [64]:
#compile model
model.compile(optimizer='adam',loss="categorical_crossentropy",metrics=['acc'])

In [65]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [67]:
model.fit(x_train,y_train,epochs=25,validation_data=(x_test,y_test))

Epoch 1/25
604/604 ━━━━━━━━━━━━━━━━━━━━ 43s 71ms/step - acc: 0.4296 - loss: 2.7333 - val_acc: 0.0560 - val_loss: 9.5532
Epoch 2/25
604/604 ━━━━━━━━━━━━━━━━━━━━ 42s 70ms/step - acc: 0.4466 - loss: 2.6437 - val_acc: 0.0564 - val_loss: 9.6893
Epoch 3/25
604/604 ━━━━━━━━━━━━━━━━━━━━ 43s 71ms/step - acc: 0.4605 - loss: 2.5651 - val_acc: 0.0594 - val_loss: 9.7899
Epoch 4/25
604/604 ━━━━━━━━━━━━━━━━━━━━ 42s 70ms/step - acc: 0.4773 - loss: 2.4803 - val_acc: 0.0550 - val_loss: 9.9043
Epoch 5/25
604/604 ━━━━━━━━━━━━━━━━━━━━ 4405s 7s/step - acc: 0.4917 - loss: 2.4072 - val_acc: 0.0569 - val_loss: 10.0052
Epoch 6/25
604/604 ━━━━━━━━━━━━━━━━━━━━ 43s 71ms/step - acc: 0.5040 - loss: 2.3482 - val_acc: 0.0564 - val_loss: 10.1192
Epoch 7/25
604/604 ━━━━━━━━━━━━━━━━━━━━ 42s 70ms/step - acc: 0.5145 - loss: 2.2807 - val_acc: 0.0525 - val_loss: 10.2345
Epoch 8/25
604/604 ━━━━━━━━━━━━━━━━━━━━ 42s 70ms/step - acc: 0.5282 - loss: 2.2209 - val_acc: 0.0550 - val_loss: 10.3130
Epoch 9/25
604/604 ━━━━━━━━━━━━━━━━━

In [86]:
model.save("Next_word_prediction_model.h5")

In [83]:
def generate_text(seed_text, length=20):
    for _ in range(length):
        encoded = tokenizer.texts_to_sequences([seed_text[-max_seq_len:]])[0]
        encoded = pad_sequences([encoded], maxlen=max_seq_len-1)
        pred = model.predict(encoded, verbose=0)
        next_char = tokenizer.index_word[np.argmax(pred)]
        seed_text += " "+ next_char
    return seed_text

print(generate_text("hat art thou that vsurp'st this"))


hat art thou that vsurp'st this the clowne shall not like a little shaking of his lunacies coarse i haue caught vp the very armour he


''